In [11]:
import numpy as np 
import os 
os.environ["KERAS_BACKEND"] = "jax"
import keras 
import pytest
from numpy import ndarray
import torch as th

In [7]:
class ContractingLayer(keras.layers.Layer):
    
    def __init__(self, name=None):
        super().__init__(name=name)
        self.convoluation_layer = keras.Sequential(
            [
                keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
            ]
        )
        self.max_pooling_layer = keras.layers.MaxPooling2D(pool_size=(2, 2))

    def call(self, x):
        y = self.convoluation_layer(x)
        z = self.max_pooling_layer(y)
        return z, y

In [ ]:
class UpsamplingLayer(keras.layers.Layer):
    
    def __init__(self, name=None):
        super().__init__(name=name)
        self.transposed_conv = keras.layers.Conv2DTranspose(32, kernel_size=3, strides=(2,2), activation='relu')
        self.concatenate_layer = keras.layers.Concatenate(axis=-1)
        self.convolution_layer = keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu")

    def call(self, x: ndarray, skip_tensor: ndarray) -> ndarray:
        x = self.transposed_conv(x)
        x = self.concatenate_layer([x, skip_tensor])
        x = self.concatenate_layer(x)
        return x

In [ ]:
class UNET(keras.Model):
    def __init__(self):
        super().__init__()

        self.cl1 = ContractingLayer()
        self.cl2 = ContractingLayer()
        self.cl3 = ContractingLayer()
        self.cl4 = ContractingLayer()
        self.cl5 = ContractingLayer()

        self.inner_conv_block = keras.Sequential(
            [
                keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            ]
        )

        self.ul1 = UpsamplingLayer()
        self.ul2 = UpsamplingLayer()
        self.ul3 = UpsamplingLayer()
        self.ul4 = UpsamplingLayer()
        self.ul5 = UpsamplingLayer()

    def call(self, x: ndarray) -> ndarray:

        x, y1 = self.cl1(x)
        x, y2 = self.cl2(x)
        x, y3 = self.cl3(x)
        x, y4 = self.cl4(x)
        x, y5 = self.cl5(x)

        x = self.inner_conv_block(x)

        x = self.ul1(x,y1)
        x = self.ul2(x,y2)
        x = self.ul3(x,y3)
        x = self.ul4(x,y4)
        x = self.ul5(x,y5)

        return self.dense(x)


In [10]:
import ipytest
ipytest.autoconfig()

def test_contracting_layer():
    input = np.zeros((1,32,32,1))
    layer = ContractingLayer()
    assert layer(input).shape == (1,14,14,128) 

def test_upsampling_layer():
    pass

def test_UNET():
    pass


ipytest.run('-vv')

======================================= test session starts =======================================
platform win32 -- Python 3.12.3, pytest-8.3.3, pluggy-1.5.0 -- c:\Users\cleme\miniconda3\envs\dsl\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\cleme\ETH\Master\DataLab\dsl-as24-challenge-3
plugins: anyio-4.2.0, typeguard-4.3.0
collecting ... collected 1 item

t_d2f13f6170454e3d897facd63deab6cf.py::test_contracting_layer PASSED                         [100%]

======================================== 1 passed in 0.17s ========================================


<ExitCode.OK: 0>

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [16]:
tensor = th.rand(10)
th.stft(tensor, 10, return_complex=False).shape

torch.Size([6, 6, 2])